In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import log_loss
import re
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import KFold,StratifiedKFold
from sklearn.metrics import accuracy_score
data_path = '/data/kaggleData/2sigma/'
import pickle

/opt/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train_df = pd.read_json(data_path + 'normalized_train.json')
test_df = pd.read_json(data_path + 'normalized_test.json')
print train_df.shape
print test_df.shape

(49352, 132)
(74659, 131)


In [3]:
features = list(test_df.columns)

In [5]:
target_num_map = {'high':0, 'medium':1, 'low':2}

train_y = np.array(train_df['interest_level'].apply(lambda x: target_num_map[x]))

KF=StratifiedKFold(train_y,5,shuffle=True,random_state = 2333)

In [6]:
cv_scores = []
i=0        
store = data_path+'lr4/'

for dev_index, val_index in KF: 
    result_dict = {}
    
    dev_set, val_set = train_df.iloc[dev_index,:] , train_df.iloc[val_index,:] 

    #filter the features
    dev_X, val_X = dev_set[features].as_matrix(), val_set[features].as_matrix()
    dev_y, val_y = train_y[dev_index], train_y[val_index]

    """
    run model
    """        
    
    lr = LogisticRegression(random_state=0)
    lr.fit(dev_X,dev_y)
    preds = lr.predict_proba(val_X)

    loss = log_loss(val_y, preds)
    
    
    #save the pickles for futures use
    pickl_file = store+'lr4-5fold-out-'+str(i)+'.pickle'
    fileObject = open(pickl_file,'wb') 
    pickle.dump(preds,fileObject)   
    fileObject.close()
    
    cv_scores.append(loss)
    i+=1
    print 'loss for the turn '+str(i)+' is '+str(loss)

print 'The mean of the cv_scores is:'
print np.mean(cv_scores)


loss for the turn 1 is 0.589774720754
loss for the turn 2 is 0.57456278532
loss for the turn 3 is 0.580800814844
loss for the turn 4 is 0.581493740643
loss for the turn 5 is 0.577715606668
The mean of the cv_scores is:
0.580869533646


/opt/anaconda2/lib/python2.7/site-packages/sklearn/linear_model/base.py:352: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


In [7]:
train_X, test_X = train_df[features].as_matrix(), test_df[features].as_matrix()

lr = LogisticRegression(random_state=0)
lr.fit(train_X,train_y)
preds = lr.predict_proba(test_X)

out_df = pd.DataFrame(preds)
out_df.columns = ["high", "medium", "low"]
out_df["listing_id"] = test_df.listing_id.values
out_df.to_json(store+'lr4-bulk-out.json')

In [8]:
cv_scores = []
i=0        
store = data_path+'lrl2C3/'

for dev_index, val_index in KF: 
    result_dict = {}
    
    dev_set, val_set = train_df.iloc[dev_index,:] , train_df.iloc[val_index,:] 

    #filter the features
    dev_X, val_X = dev_set[features].as_matrix(), val_set[features].as_matrix()
    dev_y, val_y = train_y[dev_index], train_y[val_index]

    """
    run model
    """        
    
    lr = LogisticRegression(random_state=0,C=3)
    lr.fit(dev_X,dev_y)
    preds = lr.predict_proba(val_X)

    loss = log_loss(val_y, preds)
    
    
    #save the pickles for futures use
    pickl_file = store+'lrl2C3-5fold-out-'+str(i)+'.pickle'
    fileObject = open(pickl_file,'wb') 
    pickle.dump(preds,fileObject)   
    fileObject.close()
    
    cv_scores.append(loss)
    i+=1
    print 'loss for the turn '+str(i)+' is '+str(loss)

print 'The mean of the cv_scores is:'
print np.mean(cv_scores)


loss for the turn 1 is 0.590033783899
loss for the turn 2 is 0.574193763454
loss for the turn 3 is 0.580662998331
loss for the turn 4 is 0.580520088039
loss for the turn 5 is 0.577579168464
The mean of the cv_scores is:
0.580597960437


In [9]:
train_X, test_X = train_df[features].as_matrix(), test_df[features].as_matrix()

lr = LogisticRegression(random_state=0,C=3)
lr.fit(train_X,train_y)
preds = lr.predict_proba(test_X)

out_df = pd.DataFrame(preds)
out_df.columns = ["high", "medium", "low"]
out_df["listing_id"] = test_df.listing_id.values
out_df.to_json(store+'lrl2C3-bulk-out.json')